In [1]:
import numpy as np
import pandas as pd
import cv2
import albumentations as A
from albumentations.pytorch import ToTensorV2

# To run the code just set this variable and run the notebook

In [2]:
image_path = "sample_data/t3.png"

In [3]:
import torch.nn as nn

class Generator(nn.Module):
    def __init__(self):
        super(Generator, self).__init__()
        self.main = nn.Sequential(
            nn.Conv2d(1, 256, (5,5), 1, (1, 1), bias=True),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 512, (5, 5), 1, (0, 0), bias=True),
            nn.BatchNorm2d(512),
            nn.ReLU(True),
            nn.Conv2d(512, 256, (5, 5), 1, (0,0), bias=True),
            nn.BatchNorm2d(256),
            nn.ReLU(True),
            nn.Conv2d(256, 64, (5,5), 2, (0, 0), bias=True),
            nn.BatchNorm2d(64),
            nn.ReLU(True),
            nn.Conv2d(64, 5, (5, 5), 1, (0, 0), bias=True),
            nn.Sigmoid()
        )

    def forward(self, input):
        return self.main(input)

In [4]:
import torch
generator = Generator()
generator.load_state_dict(torch.load("gen_out.pt"))
generator.eval()

Generator(
  (main): Sequential(
    (0): Conv2d(1, 256, kernel_size=(5, 5), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(256, 512, kernel_size=(5, 5), stride=(1, 1))
    (4): BatchNorm2d(512, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(512, 256, kernel_size=(5, 5), stride=(1, 1))
    (7): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (8): ReLU(inplace=True)
    (9): Conv2d(256, 64, kernel_size=(5, 5), stride=(2, 2))
    (10): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (11): ReLU(inplace=True)
    (12): Conv2d(64, 5, kernel_size=(5, 5), stride=(1, 1))
    (13): Sigmoid()
  )
)

In [5]:
import albumentations as A
from torch.utils.data import DataLoader
x_transform = A.Compose(
    [
     A.Resize(150,150),
     ToTensorV2(),
     ]
)

In [6]:
def create_string(tensor):
  chars  = ['@', '|', '>', '<', ' ']
  indexes = torch.argmax(tensor,dim=1)
  output_tensor = np.zeros((tensor.shape[0],1, 64, 64),dtype='U1')
  for k in range(tensor.shape[0]):
      for i in range(0,tensor.shape[2],1):
          for j in range(0,tensor.shape[3],1):
              index = indexes[k, i, j]
              output_tensor[k,:, i, j] = chars[index]
  return output_tensor[0,0,:,:]

generator.eval()
image = cv2.imread(image_path,cv2.IMREAD_GRAYSCALE)
image = x_transform(image=image)["image"].float()

loader = DataLoader(image,batch_size=1)


for img in loader:
  out = torch.zeros((1,1,150,150))
  out[0,0,:,:] = img
  print(out.shape)
  out = generator(out)
  s = create_string(out)
  







torch.Size([1, 1, 150, 150])


In [7]:
f_s = ""
for i in range(s.shape[0]):
    for j in range(s.shape[1]):
        f_s += s[i,j]
    f_s += "\n"


In [8]:
print(f_s)

|>>>>>>||||>>>>>@<>@>>>>>>>>>>>>>>>>>>>>><<<<>>>>||>>>>|||>>>>>>
>>>>>>||| >>>>@<@@>>>>>>>>>>>>>>>>>>>>>>>>>><><<>>>>|>>>|||>>>>>
>>>>>||| >>>>@@@>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>><<>>>>>>>>||>>>>
>>>>|||>>>>@>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>@>>>>>>>>||>>>
>>>|||>>>>>@>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>@<>>>>>>>||>>
>>|||>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>@>>>>>>>>>>
>|>|>>>>>@>@|>|||>||>|||| >>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
|>|>>>>>@@@||||||<<<|||<||| <>>>>>>>>>>>>>>>>>>>>>>>>>>> >>>>>>>
||>>>>>@@<| ||| |||||||>@ ||||>>>>>>>>>>>>>>>>>>>>>>>>>>> >>>>>>
| >>>>@@|  | ><||||||||||>  |><|<>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<>>>>@@< ||>>|||||>>||>||||  |@<|<>>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
|>>>@@@  |>||||| <  ||>|| |>> | <|<>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<>>>@<  >>><||| ><<<<<<|||  >> |  |>>>>>>>>>>>>>>>>>>>>>>>>>>>>>
<>>@ | |>><|>  <<||<<<<<<@||>>> @@ <>>>>>>>>>>>>>>>>>>>>>>>>>>>>
>>>@@ ||>< |  < <||||||<<|@||> > @  >>>>>>>>>>>>>>>>>>>>>>>>><>>
>>@@> |>@||| <|||||||||  

In [10]:
import pygame
import numpy as np

image_array = s

pygame.init()

console_width = 720
console_height = 720
console_size = (console_width, console_height)

console_screen = pygame.display.set_mode(console_size)

font_size = 8  # font size
font = pygame.font.SysFont('Arial', font_size)

for y in range(64):
    for x in range(64):
        char = image_array[y, x]
        text_surface = font.render(char, True, (255, 255, 255))  
        console_screen.blit(text_surface, (x * font_size, y * font_size))

pygame.display.update()

running = True
while running:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            running = False

pygame.quit()